In [ ]:
import mysql.connector
from mysql.connector import Error

# Database Setup and Connection
def connect_to_database():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="atul@192837465",  # Replace with your MySQL password
            database="hotel_management"
        )
        return connection
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
        return None

def initialize_database():
    connection = connect_to_database()
    if connection is None:
        return
    cursor = connection.cursor()

    # Create tables if they do not exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Rooms (
        room_id INT AUTO_INCREMENT PRIMARY KEY,
        room_number INT NOT NULL,
        room_type VARCHAR(50) NOT NULL,
        price FLOAT NOT NULL,
        status VARCHAR(20) DEFAULT 'Available'
    )""")

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Guests (
        guest_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        age INT NOT NULL,
        phone VARCHAR(15) NOT NULL,
        email VARCHAR(100) NOT NULL
    )""")

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Bookings (
        booking_id INT AUTO_INCREMENT PRIMARY KEY,
        guest_id INT NOT NULL,
        room_id INT NOT NULL,
        check_in_date DATE NOT NULL,
        check_out_date DATE NOT NULL,
        FOREIGN KEY (guest_id) REFERENCES Guests(guest_id),
        FOREIGN KEY (room_id) REFERENCES Rooms(room_id)
    )""")

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Payments (
        payment_id INT AUTO_INCREMENT PRIMARY KEY,
        booking_id INT NOT NULL,
        amount FLOAT NOT NULL,
        payment_method VARCHAR(20) NOT NULL,
        payment_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (booking_id) REFERENCES Bookings(booking_id)
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Staff (
        staff_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        age INT NOT NULL,
        post VARCHAR(50) NOT NULL,
        salary FLOAT NOT NULL,
        salary_status VARCHAR(20) DEFAULT 'Not Paid'
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Admin (
        admin_id INT AUTO_INCREMENT PRIMARY KEY,
        username VARCHAR(50) NOT NULL,
        password VARCHAR(50) NOT NULL
    )
    """)
    
    # Initialize database with a default admin user if it doesn't exist
    cursor.execute("SELECT COUNT(*) FROM Admin")
    count = cursor.fetchone()[0]
    if count == 0:  # If no admin exists, add a default admin user
        cursor.execute("INSERT INTO Admin (username, password) VALUES (%s, %s)", ('admin', 'admin123'))
        connection.commit()

    connection.commit()
    connection.close()
    print("Database initialized successfully!")

# Function to authenticate admin login
def admin_login():
    connection = connect_to_database()
    if connection is None:
        return False

    username = input("Enter admin username: ")
    password = input("Enter admin password: ")

    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Admin WHERE username = %s AND password = %s", (username, password))
    result = cursor.fetchone()

    connection.close()

    if result:
        print("Login successful! Welcome to the Hotel Management System.")
        return True
    else:
        print("Invalid username or password.")
        return False

# Function to sign up a new admin
def sign_up():
    connection = connect_to_database()
    if connection is None:
        return

    print("\n--- Admin Sign Up ---")
    username = input("Enter a new username: ")
    password = input("Enter a new password: ")

    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Admin WHERE username = %s", (username,))
    existing_admin = cursor.fetchone()

    if existing_admin:
        print("Username already exists. Please try a different username.")
        connection.close()
        return

    cursor.execute("INSERT INTO Admin (username, password) VALUES (%s, %s)", (username, password))
    connection.commit()

    print("Admin account created successfully! You can now log in.")
    connection.close()

# Function to add a new room
def add_room():
    connection = connect_to_database()
    if connection is None:
        return

    room_number = int(input("Enter room number: "))
    room_type = input("Enter room type (e.g., Luxury, Deluxe, Ultra Luxury): ")
    price = float(input("Enter room price: "))

    cursor = connection.cursor()
    cursor.execute("INSERT INTO Rooms (room_number, room_type, price) VALUES (%s, %s, %s)",
                   (room_number, room_type, price))
    connection.commit()
    print("Room added successfully!")
    connection.close()

# Function to view available rooms
def view_available_rooms():
    connection = connect_to_database()
    if connection is None:
        return

    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Rooms WHERE status = 'Available'")
    rooms = cursor.fetchall()
    
    print("\nAvailable Rooms:")
    for room in rooms:
        print(f"Room ID: {room[0]}, '\n', Room Number: {room[1]}, '\n', Room Type: {room[2]}, Price: {room[3]}")
    connection.close()

# Function to add a new guest
def add_guest():
    connection = connect_to_database()
    if connection is None:
        return

    name = input("Enter guest name: ")
    age = int(input("Enter guest age: "))
    phone = input("Enter guest phone number: ")
    email = input("Enter guest email: ")

    cursor = connection.cursor()
    cursor.execute("INSERT INTO Guests (name, age, phone, email) VALUES (%s, %s, %s, %s)",
                   (name, age, phone, email))
    connection.commit()
    print("Guest added successfully!")
    connection.close()

# Function to book a room for a guest
def book_room():
    guest_id = int(input("Enter guest ID: "))
    room_id = int(input("Enter room ID: "))
    check_in = input("Enter check-in date (YYYY-MM-DD): ")
    check_out = input("Enter check-out date (YYYY-MM-DD): ")

    connection = connect_to_database()
    if connection is None:
        return

    cursor = connection.cursor()
    cursor.execute("INSERT INTO Bookings (guest_id, room_id, check_in_date, check_out_date) VALUES (%s, %s, %s, %s)",
                   (guest_id, room_id, check_in, check_out))
    connection.commit()
    print("Room booked successfully!")
    connection.close()

# Function to make a payment for a booking
def make_payment():
    booking_id = int(input("Enter booking ID: "))
    amount = float(input("Enter payment amount: "))
    payment_method = input("Enter payment method (Cash, Card, Online): ")

    connection = connect_to_database()
    if connection is None:
        return

    cursor = connection.cursor()
    cursor.execute("INSERT INTO Payments (booking_id, amount, payment_method) VALUES (%s, %s, %s)",
                   (booking_id, amount, payment_method))
    connection.commit()
    print("Payment recorded successfully!")
    connection.close()

# Function to view all guests
def view_guests():
    connection = connect_to_database()
    if connection is None:
        return

    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Guests")
    guests = cursor.fetchall()

    print("\nGuests:")
    for guest in guests:
        print(f"Guest ID: {guest[0]}, '\n', Name: {guest[1]}, '\n', Age: {guest[2]}, '\n', Phone: {guest[3]}, '\n', Email: {guest[4]}")
    connection.close()

# Function to add a staff member
def add_staff():
    connection = connect_to_database()
    if connection is None:
        return

    name = input("Enter staff name: ")
    age = int(input("Enter staff age: "))
    post = input("Enter staff post (e.g., Housekeeping, Receptionist, Manager): ")
    salary = float(input("Enter staff salary: "))

    cursor = connection.cursor()
    cursor.execute("INSERT INTO Staff (name, age, post, salary) VALUES (%s, %s, %s, %s)",
                   (name, age, post, salary))
    connection.commit()
    print("Staff member added successfully!")
    connection.close()

# Function to view all staff members
def view_staff():
    connection = connect_to_database()
    if connection is None:
        return

    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Staff")
    staff = cursor.fetchall()

    print("\nStaff Members:")
    for staff_member in staff:
        print(f"Staff ID: {staff_member[0]}, '\n', Name: {staff_member[1]}, '\n', Age: {staff_member[2]},  '\n',Post: {staff_member[3]},  '\n',Salary: {staff_member[4]}, '\n', Status: {staff_member[5]}")
    connection.close()

# Main Menu for Hotel Management System
def main_menu():
    while True:
        print("-" * 50)
        print("\n--- Hotel Management System ---")
        print("-" * 50)
        print("1. Add Room                --")
        print("2. View Available Rooms    --")
        print("3. Add Guest               --")
        print("4. Book Room               --")
        print("5. Make Payment            --")
        print("6. View Guests             --")
        print("7. Add Staff               --")
        print("8. View Staff              --")
        print("9. Exit                    --")
        choice = input("Enter your choice: ")
        if choice == "1":
            add_room()
        elif choice == "2":
            view_available_rooms()
        elif choice == "3":
            add_guest()
        elif choice == "4":
            book_room()
        elif choice == "5":
            make_payment()
        elif choice == "6":
            view_guests()
        elif choice == "7":
            add_staff()
        elif choice == "8":
            view_staff()
        elif choice == "9":
            print("Exiting... Thank you!")
            break
        else:
            print("Invalid choice. Please try again.")

# Initialize and Run
if __name__ == "__main__":
    initialize_database()

    # Prompt for login or sign up
    print("-" * 50)  # Line above the welcome message
    print("Welcome to the Hotel Management System")
    print("-" * 50)  # Line below the welcome message
    print("1. Sign Up (Create a new admin account)")
    print("2. Log In (Existing Admin)")
    choice = input("Enter your choice: ")

    if choice == "1":
        sign_up()
    elif choice == "2":
        if admin_login():
            main_menu()
    else:
        print("Invalid choice. Exiting.")


Database initialized successfully!
--------------------------------------------------
Welcome to the Hotel Management System
--------------------------------------------------
1. Sign Up (Create a new admin account)
2. Log In (Existing Admin)
